# Get Workflow Data

This notebook uses the Firecloud API to:
1. List all the submissions of the current workspace.
2. List all the workflows of a selected submission.
3. For each workflow, parse out the cost, duration, and other metadata.

You may run step 1 of this notebook as-is. No modifications are necessary. Step 2 requires you to fill in the BigQuery IDs.

After running step 1 of this notebook, you will be prompted to select a `submissionId`. **Please copy and paste one from the displayed table, or type in "all" (without quotes) to pull all submissions**.

If your submission has several workflows, the **notebook may take several minutes to run**. This is because we must make one API call per workflow. For example, if your submission has 427 workflows, the notebook will make 427 sequential API calls which may take 4-6 minutes. 

In [ ]:
fapi.__version__

In [ ]:
import firecloud.api as fapi

In [ ]:
import firecloud.api as fapi
import os
import pandas as pd
from datetime import datetime
from ipywidgets import IntProgress
from IPython.display import display
import pandas_gbq

# Adjust these settings as needed for the dataframe displays
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 300)

## Step 1: Load submission data

In [ ]:
namespace = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']
print(f'Current workspace is {namespace}/{workspace}')

In [ ]:
r = fapi.list_submissions(namespace, workspace)
df = ''
all_submission_ids = []
if r.status_code == 200:
    data = []
    headers = ['submissionId', 'userComment', 'submissionDate', 'numComplete', 'numFailed']
    for submission in r.json():
        submissionId = submission['submissionId']
        userComment = submission['userComment']
        submissionDate = submission['submissionDate']
        workflowStatuses = submission['workflowStatuses']
        numComplete = workflowStatuses.get('Succeeded') or 0
        numFailed = workflowStatuses.get('Failed') or 0
        data.append([submissionId, userComment, submissionDate, numComplete, numFailed])
        all_submission_ids.append(submissionId)
    data.sort(key=lambda x: x[2], reverse=True) # Sort by submissionDate
    df = pd.DataFrame(data=data, columns=headers)
else:
    print(f'Error listing submissions for {namespace}/{workspace} with error {r.text}')
df

In [ ]:
submission_id = input('Please enter a submissionId from the list above, or "all" to grab all: ')
submission_id = submission_id.strip()

In [ ]:
# Helper functions

def get_workflow_metadata(submission_id, workflow_id):
    r = fapi.get_workflow_metadata(namespace, workspace, submission_id, workflow_id)
    if r.status_code == 200:
        return r.json()
    else:
        print(f'Error retrieving workflow id {workflow_id} with error {r.text}')
        
def get_workflow_entity_name(workflow):
    try:
        return workflow['workflowEntity']['entityName']
    except KeyError:
        return 'N/A'

In [ ]:
def load_one_submission(submission_id):
    r = fapi.get_submission(namespace, workspace, submission_id)
    TIME_FORMAT_STRING = '%Y-%m-%dT%H:%M:%S.%fZ'
    df = ''
    if r.status_code == 200:
        data = []
        headers = ['submissionId','workflowId','entityName', 'status', 'cost', 'duration', 'submitter']
        workflows = r.json()['workflows']
        submitter = r.json()['submitter']
        print(f'Loading workflows for submission {submission_id}:')
        progress_bar = IntProgress(min=0, max=len(workflows), description='Loading: ', style=dict(description_width='initial'))
        display(progress_bar)
        for workflow in workflows:
            cost = workflow.get('cost') or 0.0
            status = workflow['status']
            workflow_id = workflow['workflowId']
            entity_name = get_workflow_entity_name(workflow)
            wf_metadata = get_workflow_metadata(submission_id, workflow_id)
            start = datetime.strptime(wf_metadata['start'], TIME_FORMAT_STRING)
            end = datetime.strptime(wf_metadata['end'], TIME_FORMAT_STRING)
            duration = str(end-start)
            data.append([submission_id, workflow_id, entity_name, status, cost, duration, submitter])
            progress_bar.value += 1
        return pd.DataFrame(data=data, columns=headers)
    else:
        print(f'Error retrieving submission id {submission_id} with error {r.text}')
        
def load_all_submissions():
    dfs = []
    for submission_id in all_submission_ids:
        dfs.append(load_one_submission(submission_id))
    return pd.concat(dfs)
    

In [ ]:
if submission_id == 'all':
    submission_df = load_all_submissions()
else:
    submission_df = load_one_submission(submission_id)
submission_df

## Step 2: Push to BigQuery

Set `table_id` to the full destination table ID (including the project ID and dataset ID).

Ex. `table_id = 'my_project.my_dataset.my_table'`

In [ ]:
# TODO: Set table_id to the full destination table ID (including the
#       project ID and dataset ID).
table_id = 'aaa-willyn-test.1234_willyn.submissions'

In [ ]:
submission_df.to_gbq(table_id, project_id=namespace, if_exists='replace')